In [58]:
from dotenv import load_dotenv
load_dotenv('./.env')
import json
import openai
import os
from dspy import (
    OpenAI,
    OllamaLocal,
    HFModel,
    Predict,
    Signature,
    ChainOfThought,
    settings,
    predict,
    context
)
import warnings
warnings.filterwarnings('ignore')

In [34]:
openai.api_key= os.environ['OPENAI_API_KEY']

In [36]:
from rich import print

In [37]:
turbo = OpenAI(model='gpt-3.5-turbo')
settings.configure(lm=turbo)

In [40]:
signatures={"QuestionAnswering":"question -> answer",
            "SentimentClassification":"sentence -> sentiment",
            "Summarization":["document -> summary",
                             "text -> gist",
                             "long_context -> tldr"],
            "RAG":"context, question -> answer",
            "Multi-Choice":"question, choices -> reasoning, selection"}

In [41]:
signatures['QuestionAnswering']

'question -> answer'

In [42]:
# Predict class uses a signature of this form 
# only a single Predict class is required
qa_pred = Predict('question -> answer')


In [43]:
print(qa_pred)

Predict(StringSignature(question -> answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 
'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 
'Answer:', 'desc': '${answer}'})
))

In [46]:
sum_cot = ChainOfThought('document -> summary')

In [47]:
print(sum_cot)

self = Predict(StringSignature(document -> rationale, summary
    instructions='Given the fields `document`, produce the fields `summary`.'
    document = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 
'Document:', 'desc': '${document}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by 
step in order to", 'desc': '${produce the summary}. We ...', '__dspy_field_type': 'output'})
    summary = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 
'Summary:', 'desc': '${summary}'})
))

In [56]:
from dspy import(
    InputField,
    OutputField
)

class MultiClass(Signature):
    """Classift the given data into Address, Humans Name, Location, Building, Amount"""
    sentence = InputField(desc="data to be specified")
    data_type = OutputField(desc="falls in one of categories")



In [62]:
pred_class = Predict(MultiClass)
print(pred_class)

Predict(MultiClass(sentence -> data_type
    instructions='Classift the given data into Address, Humans Name, Location, Building, Amount'
    sentence = Field(annotation=str required=True json_schema_extra={'desc': 'data to be specified', 
'__dspy_field_type': 'input', 'prefix': 'Sentence:'})
    data_type = Field(annotation=str required=True json_schema_extra={'desc': 'falls in one of categories', 
'__dspy_field_type': 'output', 'prefix': 'Data Type:'})
))

In [64]:
get_json_data = Predict('required_data -> json_output')

prompt="provide me one eg of Address, Location, Human Name, building name and amount"

with context(lm=turbo):
    response = get_json_data(required_data=prompt)
    print("Response by OPENAI: ", response.json_output)

Response by OPENAI:  {
  "Address": "123 Main Street",
  "Location": "New York City",
  "Human Name": "John Doe",
  "Building Name": "Empire State Building",
  "Amount": "$100"
}

In [65]:
data= json.loads(response.json_output)

NameError: name 'json' is not defined

In [ ]:
# First lets check the ollama